In [18]:
import os

# get environment variables
cep_tiles_directory = os.environ['CEP_TILES_DIRECTORY']
water_tiles_directory = os.environ['WATER_TILES_DIRECTORY']


# Renaming JRC CEP tiles to geographic degrees

In [19]:
class TiffMatcher:
    def __init__(self, cep_tiles_directory, water_tiles_directory,is_cep_renamed=False,is_water_renamed=False):
        self.cep_tiles_directory = cep_tiles_directory
        self.water_tiles_directory = water_tiles_directory
        if is_cep_renamed == False:
            self.rename_cep_files()
        if is_water_renamed == False:
            self.rename_water_transition_tiles()

        # create a dictionary of the paired files in the two directories
        self.matches = {os.path.join(water_tiles_directory, file): os.path.join(cep_tiles_directory, file) 
                for file in os.listdir(water_tiles_directory) if file.endswith('.tiff')}

    def write_matching_files(self):
        # write the matching files to a text file
        with open('matches.txt', 'w') as f:
            for key, value in self.matches.items():
                f.write(f'{key} {value}\n')
        
    def get_new_coordinates(count, north, east, south, west, northmode, eastmode):
        # NORTH/SOUTH LOGIC
        if northmode == True:
            if count % 36 == 0:
                north += 10

        elif count % 36 == 0:
            south -= 10
            if south == 0:
                northmode = True

        # EAST/WEST LOGIC
        if eastmode:
            east += 10
            if east > 170:
                eastmode = False
                west = 180
                east = 0
        else:
            west -= 10
            if west <= 0:
                eastmode = True
                east = 0
                west = 180

        return north, east, south, west, northmode, eastmode

    def rename_cep_files(self):
        count = 0
        # south starts at 80 because we look at the top of the tile.
        north, east, south, west = 0, 0, 80, 180
        northmode, eastmode = False, False

        for count in range(648):
            prefix = f"{west}W" if not eastmode else f"{east}E"
            suffix = f"{south}S" if not northmode else f"{north}N"

            old_path = os.path.join(self.cep_tiles_directory, f"{count+1}.tiff")
            new_path = os.path.join(self.cep_tiles_directory, f"{prefix}_{suffix}.tiff")
            # rename the file
            os.rename(old_path, new_path)

            count += 1
            north, east, south, west, northmode, eastmode = self.get_new_coordinates(count, north, east, south, west, northmode, eastmode)

    def rename_water_transition_tiles(self):
        # get the list of files in the directory
        tif_files = [f for f in os.listdir(water_tiles_directory) if f.endswith('.tif')]
        
        for file in tif_files:
            # e.g transitions_0E_0Nv1_4_2021.tif -> 0E_0N.tif
            new_name = f'{file.split("_")[1]}_{file.split("_")[2].split("v")[0]}.tiff'
            os.rename(os.path.join(self.water_tiles_directory, file), os.path.join(self.water_tiles_directory, new_name))


In [20]:
tiffmatcher = TiffMatcher(water_tiles_directory, cep_tiles_directory, is_cep_renamed=True, is_water_renamed=True)
tiffmatcher.write_matching_files()
#check first match pair
for key, value in tiffmatcher.matches.items():
    print(key, value)
    break

C:\\Users\\riyad\\Documents\\Subjects\\WaterQualityContract\\JRC\\cep_oecm202302\\cep_oecm202302\\tiles\0E_0N.tiff C:\\Users\\riyad\\Documents\\Subjects\\WaterQualityContract\\GSW\\data\0E_0N.tiff
